<a href="https://colab.research.google.com/github/hsswkwk/turbo-chainsaw/blob/feature-add-anomaly-detection/notebooks/anomaly_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 異常検知（Anomaly detection）
標準的な状態や想定から逸脱しているデータや事象を特定する技術


## 正規分布に従うデータの異常検知
### 例
製品の温度をセンサーで監視している場合の異常検知
<br>
<br>
### 手法

#### 3$\sigma$法
データが正規分布に従うと仮定し、平均値から標準偏差の3倍以上離れた値を異常値とみなす方法
<br>
#### マハラノビス・タグチ法
データの各次元間の相関を考慮した距離尺度を用いて、平均値から離れた値を異常値とみなす方法
<br>
#### ホテリングの$T^2$法
マハラノビス距離を拡張した手法で、データの平均値からのずれを検定統計量として用いて異常値を検出する方法
<br>
#### 密度比推定
正常データと異常データの確率密度比を推定することで異常を検知する手法
<br>

In [ ]:
import numpy as np
from scipy.spatial.distance import mahalanobis
from scipy.stats import f
from sklearn.neighbors import KernelDensity


### 3σ法
# threshold: データの平均値から標準偏差の何倍離れたら異常値とみなすか
def three_sigma_method(data, threshold=3):
  mean = np.mean(data)
  std = np.std(data)
  lower_bound = mean - threshold * std
  upper_bound = mean + threshold * std
  if data.ndim == 1:
    anomaly_indices = np.where((data < lower_bound) | (data > upper_bound))[0]
  elif data.ndim >= 2:
    anomaly_indices = np.where(np.any((data < lower_bound) | (data > upper_bound), axis=1))[0]
  return data[anomaly_indices]

### マハラノビス距離
# マハラノビス距離: データの相関を考慮した距離尺度
# threshold_percentile: マハラノビス距離の分布におけるパーセンタイル値を超える
#                       距離を持つデータを異常値とみなす
def mahalanobis_distance_method(data, threshold_percentile=99):
  mean = np.mean(data, axis=0)
  if data.ndim == 1:
    std = np.std(data)

    # 標準偏差が0の場合、ゼロ除算を避けるために微小な値を加算
    if std == 0:
      std = 1e-6

    distances = np.abs((data - mean) / std)

    # 異常値とみなす閾値を設定
    threshold = np.percentile(distances, threshold_percentile)

    # 閾値を超えるデータを異常値として検出
    anomalies = data[distances > threshold]
  elif data.ndim >= 2:
    cov = np.cov(data, rowvar=False)
    inv_cov = np.linalg.inv(cov)

    distances = [mahalanobis(x, mean, inv_cov) for x in data]

    # 異常値とみなす閾値を設定
    threshold = np.percentile(distances, threshold_percentile)

    # 閾値を超えるデータを異常値として検出
    anomaly_indices = np.where(np.array(distances) > threshold)[0]
    anomalies = data[anomaly_indices]
  return anomalies

### ホテリングのT^2法
def hotelling_t2_method(data):
  mean = np.mean(data, axis=0)
  if data.ndim == 1:
    var = np.var(data)
    t2_values = [(x - mean)**2 / var for x in data]
    # f.ppf: F分布のパーセント点関数
    threshold = f.ppf(0.95, 1, len(data) - 1)  # 異常値とみなす閾値を設定
    anomalies = [data[i] for i, t2 in enumerate(t2_values) if t2 > threshold]
  elif data.ndim >= 2:
    mean = np.mean(data, axis=0)
    cov = np.cov(data, rowvar=False)
    inv_cov = np.linalg.inv(cov)
    t2_values = [mahalanobis(x, mean, inv_cov)**2 for x in data]
    threshold = f.ppf(0.95, data.shape[1], data.shape[0] - data.shape[1] - 1)  # 異常値とみなす閾値を設定
    anomaly_indices = np.where(np.array(t2_values) > threshold)[0]
    anomalies = data[anomaly_indices]
  return anomalies

### KLIEPによる密度比推定
# bandwidth: カーネル密度推定で使用されるカーネルの幅を制御するパラメータ
#            大きいほど推定される確率密度関数は滑らかになる
def kliep(normal_data, anomaly_data, bandwidth=0.1):
  if normal_data.ndim == 1:
    normal_data = normal_data.reshape(-1, 1)
  if anomaly_data.ndim == 1:
    anomaly_data = anomaly_data.reshape(-1, 1)
  # カーネル密度推定でデータから確率密度関数を推定する
  kde_normal = KernelDensity(bandwidth=bandwidth).fit(normal_data)
  kde_anomaly = KernelDensity(bandwidth=bandwidth).fit(anomaly_data)
  density_ratio = np.exp(kde_normal.score_samples(anomaly_data) - kde_anomaly.score_samples(anomaly_data))

  # 1e-6 を加算してゼロ除算を防ぐ
  density_ratio = np.maximum(density_ratio, 1e-6)

  # 異常度の算出
  anomaly_score = -np.log(density_ratio)

  # 閾値の設定 (例: 異常度の95%点)
  threshold = np.percentile(anomaly_score, 95)

  # 異常検知
  anomalies = anomaly_data[anomaly_score > threshold]
  return anomalies


print("1次元データの場合")
# 正常データ生成
normal_data = np.random.normal(loc=25, scale=2, size=100)

# 異常データ生成
anomaly_data = np.random.normal(loc=25, scale=2, size=100)
# 異常値の追加
anomaly_indices = [10, 20, 30]  # 異常値のインデックス
anomaly_values = [35, 15, 32]  # 異常値
anomaly_data[anomaly_indices] = anomaly_values

# 異常検知の実行
anomalies_three_sigma_method = three_sigma_method(anomaly_data)
anomalies_mahalanobis_distance_method = mahalanobis_distance_method(anomaly_data)
anomalies_hotelling_t2_method = hotelling_t2_method(anomaly_data)
anomalies_kliep = kliep(normal_data, anomaly_data)

# 結果の出力
print("================== 3σ法 =================")
print("異常値:", anomalies_three_sigma_method)
print("\n")

print("===== マハラノビス距離による異常検知 =====")
print("異常値:", anomalies_mahalanobis_distance_method)
print("\n")

print("===== ホテリングのT^2法による異常検知 ====")
print("異常値:", anomalies_hotelling_t2_method)
print("\n")

print("======== 密度比推定による異常検知 ========")
print("異常値:", anomalies_kliep)
print("\n")



print("2次元データの場合")

# 正常データ生成
normal_data2d = np.random.multivariate_normal(mean=[0, 0], cov=[[1, 0.5], [0.5, 1]], size=100)

# 異常データ生成
anomaly_data2d = np.random.multivariate_normal(mean=[0, 0], cov=[[1, 0.5], [0.5, 1]], size=100)
# 異常値の追加
anomaly_indices = [10, 20, 30]  # 異常値のインデックス
anomaly_values = [[3, 3], [-3, -3], [3, -3]]  # 異常値
anomaly_data2d[anomaly_indices] = anomaly_values

# 異常検知の実行
anomalies_three_sigma_method_2d = three_sigma_method(anomaly_data2d, threshold=2)
anomalies_mahalanobis_distance_method_2d = mahalanobis_distance_method(anomaly_data2d)
anomalies_hotelling_t2_method_2d = hotelling_t2_method(anomaly_data2d)
anomalies_kliep_2d = kliep(normal_data2d, anomaly_data2d, bandwidth=0.5)

# 結果の出力
print("================== 3σ法 =================")
print("異常値:", anomalies_three_sigma_method_2d)
print("\n")

print("===== マハラノビス距離による異常検知 =====")
print("異常値:", anomalies_mahalanobis_distance_method_2d)
print("\n")

print("===== ホテリングのT^2法による異常検知 ====")
print("異常値:", anomalies_hotelling_t2_method_2d)
print("\n")

print("======== 密度比推定による異常検知 ========")
print("異常値:", anomalies_kliep_2d)
print("\n")




1次元データの場合
================== 3σ法 =================
異常値: [35. 15.]


===== マハラノビス距離による異常検知 =====
異常値: [15.]


===== ホテリングのT^2法による異常検知 ====
異常値: [np.float64(35.0), np.float64(15.0), np.float64(32.0)]


======== 密度比推定による異常検知 ========
異常値: [[35.        ]
 [15.        ]
 [32.        ]
 [20.99479616]
 [20.89919911]]


2次元データの場合
================== 3σ法 =================
異常値: [[ 2.13848135  2.45500825]
 [ 3.          3.        ]
 [-3.         -3.        ]
 [ 3.         -3.        ]
 [ 2.71516196  1.84843097]
 [ 0.98278229  3.05596083]
 [ 2.37145929  1.40984141]]


===== マハラノビス距離による異常検知 =====
異常値: [[ 3. -3.]]


===== ホテリングのT^2法による異常検知 ====
異常値: [[ 2.13848135  2.45500825]
 [ 3.          3.        ]
 [-3.         -3.        ]
 [-2.21439899 -0.31184462]
 [ 0.06048322 -1.88858858]
 [ 3.         -3.        ]
 [ 2.71516196  1.84843097]
 [-2.04126395  0.25096997]
 [-2.16449548 -1.45698025]
 [ 0.12847085  2.13699689]
 [ 1.30552164 -1.098316  ]
 [ 0.06310474 -1.86091794]
 [ 0.98278229  3.05596083]
 [ 2.3

## 非正規データの異常検知
### 例
ウェブサイトへのアクセス数の異常検知
<br>
<br>
### 手法

#### ガンマ分布の当てはめ
データがガンマ分布に従うと仮定し、その分布から大きく外れた値を異常値とみなす方法。データがガンマ分布に従う場合、高い精度で異常を検出できるが、従わない場合、精度が低下する。
<br>

#### カイ二乗分布への当てはめ
データがカイ二乗分布に従うと仮定し、その分布から大きく外れた値を異常値とみなす方法。主に、特徴量の値が正の値を取り、歪んだ分布をしている場合に有効。<br>
<br>

#### $k$近傍法
各データポイントからk番目に近いデータポイントまでの距離を計算し、距離が大きいデータポイントを異常値と判定するアルゴリズム
<br>

#### $k$ means法
データを複数のクラスタに分割し、どのクラスタにも属さないデータポイントを異常値と判定するアルゴリズム。
<br>

#### 混合ガウス分布モデル（Gaussian Mixture Model, GMM）
データが複数のガウス分布（正規分布）の混合で表現できると仮定し、低確率なデータ点を異常値とみなす方法。
<br>

#### One-Class SVM
正常データのみを用いて、正常データの領域を学習するアルゴリズム。 学習した領域から外れたデータは異常値と判定する。
<br>

#### [密度比推定](#scrollTo=1dE3Gampih0k&line=1&uniqifier=1)
<br>

#### 孤立フォレスト（Isolation Forest）
データポイントをランダムに分割していくことで、異常値を孤立させるアルゴリズム。
<br>

#### Local Outlier Factor (LOF)
データポイントの局所的な密度を計算し、密度が低いデータポイントを異常値と判定するアルゴリズム。
<br>



In [ ]:
import numpy as np
import scipy.stats as stats
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor


def fitting_gamma_distribution(data):
  # データにガンマ分布を当てはめる
  shape, loc, scale = stats.gamma.fit(data)

  # 異常度を計算する
  anomaly_scores = 1 / stats.gamma.pdf(data, shape, loc, scale)

  # 閾値を設定する
  threshold = np.mean(anomaly_scores) + 3 * np.std(anomaly_scores)

  # 異常値を検出する
  anomalies = data[anomaly_scores > threshold]

  return anomalies


def fitting_chi_square_distribution(data):
  # データにカイ二乗分布を当てはめる
  df, loc, scale = stats.chi2.fit(data)

  # 異常度を計算する (確率密度関数の逆数を使用)
  anomaly_scores = 1 / stats.chi2.pdf(data, df, loc, scale)

  # 閾値を設定する (上位5%点を閾値にする)
  threshold = stats.chi2.ppf(0.95, df, loc, scale)

  # 異常値を検出する
  anomalies = data[anomaly_scores > threshold]

  return anomalies


def k_nearest_neighbors(data, k):
  # k近傍の計算
  knn = NearestNeighbors(n_neighbors=k)
  knn.fit(data)
  distances, indices = knn.kneighbors(data)

  # 異常度の算出
  anomaly_score = distances[:, -1]

  # 異常値の判定
  anomaly_index = np.where(anomaly_score > threshold)

  # 異常値を検出する
  anomalies = data[anomaly_index]

  return anomalies


def k_means(data, k):
  # クラスタリングの実行
  kmeans = KMeans(n_clusters=k)
  kmeans.fit(data)
  labels = kmeans.labels_

  # 異常値の判定
  anomaly_index = np.where(labels == -1)

  # 異常値を検出する
  anomalies = data[anomaly_index]

  return anomalies


def gaussian_mixture_model(data):
  # GMMモデルの構築 (コンポーネント数は3)
  gmm = GaussianMixture(n_components=3)

  # データを用いてモデルを学習
  gmm.fit(data)

  # 各データ点の確率密度を計算
  densities = gmm.score_samples(data)
  # 異常度は確率密度の負の対数で表されることが多い
  anomaly_scores = -densities

  # 閾値を設定 (例：確率密度の下位5%点を閾値にする)
  threshold = np.percentile(anomaly_scores, 5)

  # 異常値を検出
  anomalies = data[anomaly_scores > threshold]

  return anomalies


def one_class_svm(data):
  # モデルの学習
  model = OneClassSVM()
  model.fit(data)

  # 異常度の算出
  anomaly_score = model.decision_function(data)

  # 異常値の判定
  anomaly_index = np.where(anomaly_score < 0)

  # 異常値を検出する
  anomalies = data[anomaly_index]

  return anomalies


def isolation_forest(data):
  # モデルの学習
  model = IsolationForest()
  model.fit(data)

  # 異常度の算出
  anomaly_score = model.decision_function(data)

  # 異常値の判定
  anomaly_index = np.where(anomaly_score < 0)

  # 異常値を検出する
  anomalies = data[anomaly_index]

  return anomalies


def local_outlier_factor(data):
  # モデルの学習
  model = LocalOutlierFactor()
  anomaly_score = model.fit_predict(data)

  # 異常値の判定
  anomaly_index = np.where(anomaly_score == -1)

  # 異常値を検出する
  anomalies = data[anomaly_index]

  return anomalies


def power_law_distribution(alpha, xmin, xmax, size):
    """べき乗則に従う乱数を生成する関数

    Args:
        alpha (float): べき乗則の指数
        xmin (float): 最小値
        xmax (float): 最大値
        size (int): 生成する乱数の数

    Returns:
        numpy.ndarray: べき乗則に従う乱数
    """

    u = np.random.uniform(size=size)
    return (xmin ** (1 - alpha) + (xmax ** (1 - alpha) - xmin ** (1 - alpha)) * u) ** (1 / (1 - alpha))


# 正常データの生成
normal_data = power_law_distribution(alpha=2, xmin=1, xmax=100, size=1000)

# 異常データの生成
outliers = np.random.uniform(low=2*normal_data.max(), high=3*normal_data.max(), size=10)
anomaly_data = np.concatenate([normal_data, outliers])

all_data = np.concatenate([normal_data, anomaly_data])

#### ガンマ分布の当てはめ
anomalies = fitting_gamma_distribution(all_data)
print("============ ガンマ分布の当てはめ ==========")
print("異常値:", anomalies)
print("\n")

#### カイ二乗分布への当てはめ
anomalies = fitting_chi_square_distribution(all_data)
print("========= カイ二乗分布への当てはめ =========")
print("異常値:", anomalies)
print("\n")

#### k近傍法
anomalies = k_nearest_neighbors(all_data, k)
print("================== k近傍法 =================")
print("異常値:", anomalies)
print("\n")

#### k means法
anomalies = k_means(all_data, k)
print("================= k means法 ================")
print("異常値:", anomalies)
print("\n")

#### 混合ガウス分布モデル（Gaussian Mixture Model, GMM）
anomalies = gaussian_mixture_model(all_data)
print("============ 混合ガウス分布モデル ==========")
print("異常値:", anomalies)
print("\n")

#### One-Class SVM
anomalies = isolation_forest(all_data)
print("=============== One-Class SVM ==============")
print("異常値:", anomalies)
print("\n")

#### 密度比推定
anomalies = kliep(normal_data, anomaly_data)
print("================ 密度比推定 ================")
print("異常値:", anomalies)
print("\n")

#### 孤立フォレスト (Isolation Forest)
anomalies = isolation_forest(all_data)
print("=============== 孤立フォレスト ==============")
print("異常値:", anomalies)
print("\n")

#### Local Outlier Factor (LOF)
anomalies = local_outlier_factor(all_data)
print("============ Local Outlier Factor ===========")
print("異常値:", anomalies)
print("\n")


## 不要次元のある次元データの異常検知
### 例
顧客の購買データを用いた異常検知
<br>
<br>
### 次元削減を用いる手法

#### 主成分分析（PCA）
データの分散を最大化するように新しい軸を定義し、その軸にデータを射影することで次元を削減する手法。異常なデータ点は、主成分空間において、正常なデータ点から離れた位置に配置される傾向がある。
<br>

#### 確率的主成分分析（Probabilistic PCA）
PCAを確率モデルとして拡張した手法。データにノイズが含まれる場合に、よりロバストな結果が得られる。（＝データにノイズや外れ値が含まれていても、分析結果が大きく影響を受けにくい）
<br>

#### カーネル主成分分析（Kernel PCA）
非線形な関係を持つデータに対して、カーネル関数を使用して高次元空間に写像し、その空間でPCAを行うことで次元を削減する手法。
<br>

#### 因子分析（Factor Analysis）
観測変数の背後にある潜在変数を推定し、それらを用いて次元を削減する手法。
<br>

#### 独立成分分析（ICA）
観測変数を、統計的に独立な成分に分解することで次元を削減する手法。
<br>

#### t-distributed Stochastic Neighbor Embedding (t-SNE)
高次元データを低次元空間に埋め込む際に、データの局所的な構造を保持する様に設計された手法。異常検知では、正常なデータ点と異常なデータ点が、低次元空間において明確に分離されることが期待する。
<br>

#### Uniform Manifold Approximation and Projection (UMAP)
高次元データを低次元空間に埋め込む際に、データのトポロジカルな構造を保持する様に設計された手法です。t-SNEと同様に、異常検知では、正常なデータ点と異常なデータ点が、低次元空間において明確に分離されることを期待する。
<br>

### 特徴量選択を用いる手法
#### フィルター法
各特徴量と目的変数の間の相関や相互情報量などを用いて、重要度の低い特徴量を除去する手法。
<br>

#### ラッパー法
特徴量のサブセットを選択し、そのサブセットを用いて学習したモデルの性能を評価することで、最適な特徴量を選択する手法。
<br>

#### 埋め込み法
モデルの学習過程で特徴量選択を行う手法。LASSO回帰や決定木などが用いられる。
<br>

### その他の手法
#### [One-Class SVM](#scrollTo=FMOqmjNLiot5&line=33&uniqifier=1)
正常なデータのみを用いて、正常データの領域を学習し、その領域から外れたデータを異常と判定する手法。
<br>

#### [孤立フォレスト（Isolation Forest）](#scrollTo=FMOqmjNLiot5&line=33&uniqifier=1)
<br>

#### [Local Outlier Factor (LOF)](#scrollTo=FMOqmjNLiot5&line=33&uniqifier=1)
<br>






In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA


#### 正常データの生成
# 顧客ID、年齢、性別、購入金額、購入頻度、不要な次元（ランダムな数値）を生成
num_customers = 100  # 顧客数
customer_ids = np.arange(1, num_customers + 1)
ages = np.random.randint(20, 60, size=num_customers)
genders = np.random.choice(['Male', 'Female'], size=num_customers)
purchase_amounts = np.random.randint(100, 10000, size=num_customers)
purchase_frequencies = np.random.randint(1, 10, size=num_customers)
unnecessary_dimension = np.random.rand(num_customers)

# データフレームを作成
normal_data = pd.DataFrame({
  'CustomerID': customer_ids,
  'Age': ages,
  'Gender': genders,
  'PurchaseAmount': purchase_amounts,
  'PurchaseFrequency': purchase_frequencies,
  'UnnecessaryDimension': unnecessary_dimension  # 不要な次元
})

#### 異常データの生成
anomaly_data = normal_data.copy()

# 異常値を挿入するインデックスをランダムに選択
# 例：顧客の10%を異常値とする
anomaly_indices = np.random.choice(anomaly_data.index, size=int(num_customers * 0.1), replace=False)

# 選択したインデックスのデータに異常値を代入
# 例：PurchaseAmountを極端に大きくする
anomaly_data.loc[anomaly_indices, 'PurchaseAmount'] = anomaly_data.loc[anomaly_indices, 'PurchaseAmount'] * np.random.uniform(100, 200, size=len(anomaly_indices))


def pca(normal_data, anomaly_data):
  # 不要な次元を除外
  features = ['Age', 'PurchaseAmount', 'PurchaseFrequency', 'UnnecessaryDimension']  # Genderはカテゴリカルデータなので除外
  X_normal = normal_data[features].values
  X_anomaly = anomaly_data[features].values

  # データを標準化
  scaler = StandardScaler()
  X_normal_scaled = scaler.fit_transform(X_normal)
  X_anomaly_scaled = scaler.transform(X_anomaly)

  # PCAモデルを学習
  pca = PCA(n_components=2)  # 次元数を2に削減
  pca.fit(X_normal_scaled)  # 正常データでモデルを学習

  # 異常度を計算
  normal_scores = pca.transform(X_normal_scaled)
  anomaly_scores = pca.transform(X_anomaly_scaled)

  # 再構成誤差を異常度として使用
  reconstruction_error_normal = np.sum(np.square(X_normal_scaled - pca.inverse_transform(normal_scores)), axis=1)
  reconstruction_error_anomaly = np.sum(np.square(X_anomaly_scaled - pca.inverse_transform(anomaly_scores)), axis=1)

  # 閾値を設定 (例: 再構成誤差の95%点)
  threshold = np.percentile(reconstruction_error_normal, 95)

  # 異常を検出
  anomalies = anomaly_data[reconstruction_error_anomaly > threshold]

  return anomalies


def probabilistic_pca(normal_data, anomaly_data):
  # 不要な次元を除外
  features = ['Age', 'PurchaseAmount', 'PurchaseFrequency', 'UnnecessaryDimension']  # Genderはカテゴリカルデータなので除外
  X_normal = normal_data[features].values
  X_anomaly = anomaly_data[features].values

  # データを標準化
  scaler = StandardScaler()
  X_normal_scaled = scaler.fit_transform(X_normal)
  X_anomaly_scaled = scaler.transform(X_anomaly)

  # 確率的PCAモデルを学習
  ppca = PCA(n_components=2, svd_solver='full')  # 次元数を2に削減, svd_solver='full'で確率的PCAを指定
  ppca.fit(X_normal_scaled)  # 正常データでモデルを学習

  # 異常度を計算 (平均対数尤度)
  anomaly_scores = -ppca.score_samples(X_anomaly_scaled)

  # 閾値を設定 (例: 平均対数尤度の5%点)
  threshold = np.percentile(anomaly_scores, 5)

  # 異常を検出
  anomalies = anomaly_data[anomaly_scores < threshold]  # 平均対数尤度が低いデータが異常

  return anomalies


def kernel_pca(normal_data, anomaly_data):
  # 不要な次元を除外
  features = ['Age', 'PurchaseAmount', 'PurchaseFrequency', 'UnnecessaryDimension']  # Genderはカテゴリカルデータなので除外
  X_normal = normal_data[features].values
  X_anomaly = anomaly_data[features].values

  # データを標準化
  scaler = StandardScaler()
  X_normal_scaled = scaler.fit_transform(X_normal)
  X_anomaly_scaled = scaler.transform(X_anomaly)

  # Kernel PCAモデルを学習
  kpca = KernelPCA(n_components=2, kernel='rbf')  # 次元数を2に削減, kernel='rbf'でRBFカーネルを指定
  kpca.fit(X_normal_scaled)  # 正常データでモデルを学習

  # 異常度を計算 (再構成誤差)
  normal_scores = kpca.transform(X_normal_scaled)
  anomaly_scores = kpca.transform(X_anomaly_scaled)

  # 再構成誤差を計算 (近似)
  reconstruction_error_normal = np.sum(np.square(X_normal_scaled - kpca.inverse_transform(normal_scores)), axis=1)
  reconstruction_error_anomaly = np.sum(np.square(X_anomaly_scaled - kpca.inverse_transform(anomaly_scores)), axis=1)

  # 閾値を設定 (例: 再構成誤差の95%点)
  threshold = np.percentile(reconstruction_error_normal, 95)

  # 異常を検出
  anomalies = anomaly_data[reconstruction_error_anomaly > threshold]  # 再構成誤差が閾値を超えるデータが異常

  return anomalies


#### 主成分分析（PCA）
anomalies = pca(normal_data, anomaly_data)
print("============ 主成分分析 ===========")
print("異常値:", anomalies)
print("\n")

#### 確率的主成分分析（Probabilistic PCA）
anomalies = probabilistic_pca(normal_data, anomaly_data)
print("========= 確率的主成分分析 ========")
print("異常値:", anomalies)
print("\n")

#### カーネル主成分分析（Kernel PCA）
anomalies = kernel_pca(normal_data, anomaly_data)
print("======== カーネル主成分分析 ========")
print("異常値:", anomalies)
print("\n")

#### 因子分析（Factor Analysis）

#### 独立成分分析（ICA）

#### t-distributed Stochastic Neighbor Embedding (t-SNE)

#### Uniform Manifold Approximation and Projection (UMAP)

#### フィルター法

#### ラッパー法

#### 埋め込み法

#### One-Class SVM

#### 孤立フォレスト（Isolation Forest）

#### Local Outlier Factor (LOF)



## 入出力関係のあるデータの異常検知
### 例
製造工程における品質管理


## 時系列データの異常検知
### 例
サーバーのCPU使用率の異常検知


## 変数間に関係があるデータの異常検知
### 例
クレジットカードの不正利用検知
